In [2]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('../execution_results/execute_sw.csv')


In [3]:
df.head()

,Instance,Algorithme,Seed,Score,Lambda,MaxEval,CPU-Used-Time (ms)
0,instances/atsp_rand_50_75.txt,sampled_walk_swap,50,2370,5,100000,347.998
1,instances/atsp_rand_50_75.txt,sampled_walk_swap,50,2370,9,100000,191.641
2,instances/atsp_rand_50_75.txt,sampled_walk_swap,51,2616,5,100000,343.734
3,instances/atsp_rand_50_75.txt,sampled_walk_swap,51,2683,9,100000,187.370
4,instances/atsp_rand_50_75.txt,sampled_walk_swap,52,2602,5,100000,340.542


In [20]:
neighbor_relations = ["swap","2opt"]
versus = ('swap', '2opt')
instances = df['Instance'].unique()
lambdas = df["Lambda"].unique()


In [11]:
def get_score(instance,neighbor_relation,seed,ld):
    filtre = (
    (df['Instance'] == instance) &
    (df['Algorithme'] == f"sampled_walk_{neighbor_relation}") &
    (df['Seed'] == seed) & 
    (df["Lambda"] == ld)
    )
    r=df[filtre]
    if r.size != 0:
        return r.iloc[0][3]
    else:
        print(instance,neighbor_relation,seed)
    

In [13]:
def gagne(instance,seed, vs,ld):
    #print("vs:",vs)
    s1 = get_score(instance,vs[0],seed,ld)
    s2 = get_score(instance,vs[1],seed,ld)
    return s1 < s2

In [15]:
get_score("instances/atsp_rand_50_75.txt","swap",50,9)

2370

In [16]:
get_score("instances/atsp_rand_50_75.txt","2opt",50,9)

2431

In [18]:
gagne("instances/atsp_rand_50_75.txt",50, ("swap","2opt"),9)

True

In [23]:
result_tuple={"instances" : instances, "swap_vs_2opt" : []}
for instance in instances:
    
    r=[0,0]
    for ld in lambdas:
        for seed in df["Seed"].unique():
            if gagne(instance,seed, versus,ld):
                r[0]+=1
            else:
                r[1]+=1
            
    result_tuple["swap_vs_2opt"].append((r[0],r[1]))
            
            #print(r)
                
result_tuple = pd.DataFrame(result_tuple)
result_tuple

,instances,swap_vs_2opt
0,instances/atsp_rand_50_75.txt,"(10, 10)"
1,instances/atsp_rand_50_100.txt,"(13, 7)"
2,instances/atsp_rand_60_50.txt,"(10, 10)"
3,instances/atsp_rand_60_100.txt,"(8, 12)"
4,instances/atsp_rand_70_55.txt,"(10, 10)"
5,instances/atsp_rand_70_100.txt,"(9, 11)"
6,instances/atsp_rand_80_70.txt,"(12, 8)"
7,instances/atsp_rand_80_100.txt,"(6, 14)"


In [25]:
result={"instances" : instances, "swap_vs_2opt" : []}
for instance in instances:
    
    r=[0,0]
    for ld in lambdas:
        for seed in df["Seed"].unique():
            if gagne(instance,seed, versus,ld):
                r[0]+=1
            else:
                r[1]+=1
            
    result["swap_vs_2opt"].append(r[0])
            
            #print(r)
                
result = pd.DataFrame(result)
result

,instances,swap_vs_2opt
0,instances/atsp_rand_50_75.txt,10
1,instances/atsp_rand_50_100.txt,13
2,instances/atsp_rand_60_50.txt,10
3,instances/atsp_rand_60_100.txt,8
4,instances/atsp_rand_70_55.txt,10
5,instances/atsp_rand_70_100.txt,9
6,instances/atsp_rand_80_70.txt,12
7,instances/atsp_rand_80_100.txt,6


In [31]:
mean_swap = result["swap_vs_2opt"].sum()/8
mean_swap

9.75

In [30]:
mean_2opt = (8*20 -result["swap_vs_2opt"].sum() ) /8
mean_2opt

10.25